In [1]:
#pip install transformers torch torchvision pillow

Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to encode text and image
def encode_text_and_image(text, image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(text=[text], images=image, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        text_embedding = outputs.text_embeds
        image_embedding = outputs.image_embeds

    return text_embedding, image_embedding

# Function to find the closest match in a folder
def find_best_match(text_query, image_folder):
    best_match = None
    best_score = -1

    for img_name in os.listdir(image_folder):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder, img_name)
            
            text_emb, img_emb = encode_text_and_image(text_query, image_path)
            score = torch.cosine_similarity(text_emb, img_emb).item()
            
            if score > best_score:
                best_score = score
                best_match = img_name

    return best_match, best_score

In [8]:
if __name__ == "__main__":
    image_folder = "lost_items"  # Folder with lost item images
    text_query = "spex"  # User’s search description

    # Find the best match
    best_match, score = find_best_match(text_query, image_folder)

    # Show result
    if best_match:
        print(f"Best Match: {best_match} (Similarity Score: {score:.4f})")
        Image.open(os.path.join(image_folder, best_match)).show()
    else:
        print("No match found!")

Best Match: img3.jpg (Similarity Score: 0.2844)
